In [1]:
using MixedLayerModel
using FileIO
using DifferentialEquations
using Sundials
using Revise
using Plots

using Logging: global_logger
using TerminalLoggers: TerminalLogger
global_logger(TerminalLogger());

In [12]:
include("mlm_solve_funcs.jl")

# set-up basic 400 ppm simulation
par = basic_params();
par.etype = bflux();
path = "./output/bflux_timeevolve/";

# run to steady-state
u0, sol = run_mlm(par);

# get output
code = sol.retcode;
uf = sol.u[end];
du = zeros(4);
mlm(du, uf, par, 0.0);
zi,hM,qM,SST = uf;
zb = calc_LCL(zi,hM,qM);
println(code);
println(uf);
println(du ./ uf);

# save to file
output = Dict("code" => code, "p"=>par, "u0" => u0, "uf" => uf, "du/u" => du./uf, 
"we" => we(uf,par,par.etype), "zb" => zb, "zc" => zi-zb,
"RHsurf" => RH(0.0, hM, qM), "LHF" => calc_LHF(uf,par), "SHF" => calc_SHF(uf,par),
"ΔR" => calc_cloudtop_RAD(uf,par,par.rtype), "OHU" => calc_OHU(uf,par,par.stype));
save(path*"co2_400.jld2", output)

# run steady-state
u0, solss = run_mlm_ss(par);
uf = solss.u;
println(uf);
# plot
t = sol.t / 3600.0 / 24.0
zi = getindex.(sol.u,1)
hM = getindex.(sol.u,2) * 1e-3
qtM = getindex.(sol.u,3) * 1e3
plot(t, [zi, hM, qtM], marker="o-", label=["zi(t) [m]" "hM(t) [kJ/kg]" "qtM(t) [g/kg]"], legend=:top, layout=(3,1))
hline!([uf[1]], label="SS, "*string(round(uf[1],digits=1))*" [m]", subplot=1)
hline!([uf[2]*1e-3], label="SS, "*string(round(uf[2]*1e-3,digits=1))*" [kJ/kg]", subplot=2)
hline!([uf[3]*1e3], label="SS, "*string(round(uf[3]*1e3,digits=2))*" [g/kg]", subplot=3)
title!("400 (ppm)", subplot=1)
xaxis!("t (days)", subplot=3)
mkpath(replace(path, "output" => "figures"))
savefig(replace(path, "output" => "figures")*"co2_400.png")

ODE   0%|                                               |  ETA: N/A
ODE  21%|█████████▊                                     |  ETA: 0:00:06
ODE  42%|███████████████████▋                           |  ETA: 0:00:03
ODE  62%|█████████████████████████████▍                 |  ETA: 0:00:02
ODE  83%|███████████████████████████████████████▏       |  ETA: 0:00:01


  6.913291 seconds (33.46 M allocations: 1.274 GiB, 2.37% gc time, 41.70% compilation time)


ODE 100%|███████████████████████████████████████████████| Time: 0:00:04


Success
[914.0844994859599, 312455.74758193997, 0.008769258507783524, 290.0]
[-8.660179815163548e-10, -2.6027569727870523e-12, 6.916097854286773e-11, 0.0]
euler, dt=4 hrs


ODE   0%|                                               |  ETA: N/A
ODE  62%|█████████████████████████████▍                 |  ETA: 0:00:02
ODE 100%|███████████████████████████████████████████████| Time: 0:00:04


 12.208718 seconds (50.64 M allocations: 2.854 GiB, 3.13% gc time, 65.57% compilation time)
[914.0844994859599, 312455.74758193997, 0.008769258507783524, 290.0]


┌ Warning: Skipped marker arg o-.
└ @ Plots /Users/claresinger/.julia/packages/Plots/HcxwM/src/args.jl:1178


In [13]:
include("mlm_solve_funcs.jl")

# set co2
newCO2 = 1200.0

# load initial condition from file
par = basic_params();
par.etype = bflux();
path = "./output/bflux_timeevolve/";
output = load(path*"co2_400.jld2");
u0 = output["uf"];
du = zeros(4);
mlm(du, u0, par, 0.0);
println(u0);
println(du ./ u0);

# increase CO2, let SST evolve and check cloud changes
par.CO2 = newCO2;
u0, sol = run_mlm_from_init(u0, par, dt=3600.0*2.0, tspan=(0.0,3600.0*24.0*15.0));

# get output
code = sol.retcode;
uf = sol.u[end];
du = zeros(4);
mlm(du, uf, par, 0.0);
zi,hM,qM,SST = uf;
zb = calc_LCL(zi,hM,qM);
println(code);
println(uf);
println(du ./ uf);

# save to file
output = Dict("code" => code, "p" => par, "u0" => u0, "uf" => uf, "du/u" => du./uf, 
"we" => we(uf,par,par.etype), "zb" => zb, "zc" => zi-zb,
"RHsurf" => RH(0.0, hM, qM), "LHF" => calc_LHF(uf,par), "SHF" => calc_SHF(uf,par),
"ΔR" => calc_cloudtop_RAD(uf,par,par.rtype), "OHU" => calc_OHU(uf,par,par.stype))
save(path*"co2_upstep_fixSST_"*string(Int(newCO2))*".jld2", output)

# run steady-state
u0, solss = run_mlm_ss_from_init(u0, par, dt=3600.0*2.0, tspan=3600.0*24.0*15.0);
uf = solss.u;
println(uf);

# plot
t = sol.t / 3600.0 / 24.0
zi = getindex.(sol.u,1)
hM = getindex.(sol.u,2) * 1e-3
qtM = getindex.(sol.u,3) * 1e3
plot(t, [zi, hM, qtM], marker="o-", 
    label=["zi(t) [m]" "hM(t) [kJ/kg]" "qtM(t) [g/kg]"], legend=:top, layout=(3,1))
hline!([uf[1]], label="SS, "*string(round(uf[1],digits=1))*" [m]", subplot=1)
hline!([uf[2]*1e-3], label="SS, "*string(round(uf[2]*1e-3,digits=1))*" [kJ/kg]", subplot=2)
hline!([uf[3]*1e3], label="SS, "*string(round(uf[3]*1e3,digits=2))*" [g/kg]", subplot=3)
title!("fixed SST @ "*string(newCO2)*" (ppm)", subplot=1)
xaxis!("t (days)", subplot=3)
mkpath(replace(path, "output" => "figures"))
savefig(replace(path, "output" => "figures")*"co2_upstep_fixSST_"*string(Int(newCO2))*".png")

[914.0844994859599, 312455.74758193997, 0.008769258507783524, 290.0]
[1.1839986372918933e-6, -1.121257643929456e-7, -2.259694579743309e-6, 0.0]


ODE   0%|                                               |  ETA: N/A
ODE   6%|██▋                                            |  ETA: 0:00:11
ODE  11%|█████▎                                         |  ETA: 0:00:09
ODE  17%|███████▉                                       |  ETA: 0:00:08
ODE  22%|██████████▌                                    |  ETA: 0:00:08
ODE  28%|█████████████                                  |  ETA: 0:00:07
ODE  33%|███████████████▋                               |  ETA: 0:00:06
ODE  39%|██████████████████▎                            |  ETA: 0:00:06
ODE  44%|████████████████████▉                          |  ETA: 0:00:05
ODE  50%|███████████████████████▌                       |  ETA: 0:00:05
ODE  56%|██████████████████████████▏                    |  ETA: 0:00:04
ODE  61%|████████████████████████████▊                  |  ETA: 0:00:04
ODE  67%|███████████████████████████████▍               |  ETA: 0:00:03
ODE  72%|██████████████████████████████████             |  ETA: 0:00

  8.963279 seconds (67.10 M allocations: 2.043 GiB, 2.61% gc time)
Success
[627.6912200912502, 315410.82034858613, 0.009745472658789708, 290.0]
[-1.5060027939684273e-10, -5.205543191028687e-13, 5.199211878090548e-12, 0.0]
euler

ODE 100%|███████████████████████████████████████████████| Time: 0:00:08
ODE 100%|███████████████████████████████████████████████| Time: 0:00:08


ODE   0%|                                               |  ETA: N/A
ODE  17%|███████▉                                       |  ETA: 0:00:08
ODE  33%|███████████████▋                               |  ETA: 0:00:06
ODE  50%|███████████████████████▌                       |  ETA: 0:00:05
ODE  67%|███████████████████████████████▍               |  ETA: 0:00:03


  7.438923 seconds (55.49 M allocations: 1.690 GiB, 2.47% gc time)
[627.7576769687556, 315410.9348021278, 0.009745436727772004, 290.0]


ODE 100%|███████████████████████████████████████████████| Time: 0:00:07
┌ Warning: Skipped marker arg o-.
└ @ Plots /Users/claresinger/.julia/packages/Plots/HcxwM/src/args.jl:1178


In [3]:
include("mlm_solve_funcs.jl")

# set co2
newCO2 = 1200.0

# load initial condition from file
par = basic_params();
par.etype = bflux();
path = "./output/bflux_timeevolve/";
output = load(path*"co2_400.jld2");
u0 = output["uf"];
OHU = output["OHU"];
du = zeros(4);
mlm(du, u0, par, 0.0);
println(u0);
println(du ./ u0);

# set OHU, let SST evolve and check cloud changes
par.CO2 = newCO2;
par.Hw = 1;
par.OHU = OHU;
par.stype = varSST();
u0, sol = run_mlm_from_init(u0, par, dt=3600.0*0.1, tspan=(0.0,3600.0*24.0*10.0));

# get output
code = sol.retcode;
uf = sol.u[end];
du = zeros(4);
mlm(du, uf, par, 0.0);
zi,hM,qM,SST = uf;
zb = calc_LCL(zi,hM,qM);
println(code);
println(uf);
println(du ./ uf);

# save to file
output = Dict("code" => code, "p" => par, "u0" => u0, "uf" => uf, "du/u" => du./uf, 
"we" => we(uf,par,par.etype), "zb" => zb, "zc" => zi-zb,
"RHsurf" => RH(0.0, hM, qM), "LHF" => calc_LHF(uf,par), "SHF" => calc_SHF(uf,par),
"ΔR" => calc_cloudtop_RAD(uf,par,par.rtype), "OHU" => calc_OHU(uf,par,par.stype))
save(path*"co2_upstep_"*string(Int(newCO2))*".jld2", output)

# run steady-state
u0, solss = run_mlm_ss_from_init(u0, par, dt=3600.0*0.1, tspan=3600.0*24.0*10.0);
uf = solss.u;
println(uf);

# plot
t = sol.t / 3600.0 / 24.0
zi = getindex.(sol.u,1)
hM = getindex.(sol.u,2) * 1e-3
qtM = getindex.(sol.u,3) * 1e3
SST = getindex.(sol.u,4)
plot(t, [zi, hM, qtM, SST], marker="o-", 
    label=["zi(t) [m]" "hM(t) [kJ/kg]" "qtM(t) [g/kg]" "SST(t) [K]"], legend=:top, layout=(4,1))
hline!([uf[1]], label="SS, "*string(round(uf[1],digits=1))*" [m]", subplot=1)
hline!([uf[2]*1e-3], label="SS, "*string(round(uf[2]*1e-3,digits=1))*" [kJ/kg]", subplot=2)
hline!([uf[3]*1e3], label="SS, "*string(round(uf[3]*1e3,digits=2))*" [g/kg]", subplot=3)
hline!([uf[4]], label="SS, "*string(round(uf[4],digits=1))*" [K]", subplot=4)
title!("slab-ocean @ "*string(newCO2)*" (ppm)", subplot=1)
xaxis!("t (days)", subplot=4)
mkpath(replace(path, "output" => "figures"))
savefig(replace(path, "output" => "figures")*"co2_upstep_"*string(Int(newCO2))*".png")

[914.0844994859599, 312455.74758193997, 0.008769258507783524, 290.0]
[1.1839986372918933e-6, -1.121257643929456e-7, -2.259694579743309e-6, 0.0]


ODE   0%|                                               |  ETA: N/A
ODE   2%|█                                              |  ETA: 0:05:52
ODE   4%|██                                             |  ETA: 0:05:13
ODE   6%|███                                            |  ETA: 0:04:49
ODE   8%|███▉                                           |  ETA: 0:04:34
ODE  10%|████▉                                          |  ETA: 0:04:20
ODE  12%|█████▉                                         |  ETA: 0:04:08
ODE  15%|██████▉                                        |  ETA: 0:03:56
ODE  17%|███████▉                                       |  ETA: 0:03:47
ODE  19%|████████▊                                      |  ETA: 0:03:37
ODE  21%|█████████▊                                     |  ETA: 0:03:28
ODE  23%|██████████▊                                    |  ETA: 0:03:19
ODE  25%|███████████▊                                   |  ETA: 0:03:11
ODE  27%|████████████▊                                  |  ETA: 0:03

 78.904588 seconds (457.57 M allocations: 14.133 GiB, 1.99% gc time, 3.28% compilation time)
Success
[308.81739933127403, 687107.106657062, 0.130694473172715, 332.7862285876953]
[-1.3624157047880416e-6, 1.2778397550783975e-6, 2.5435448163686336e-6, 1.0601548073321662e-7]
euler

ODE 100%|███████████████████████████████████████████████| Time: 0:01:16


ODE   0%|                                               |  ETA: N/A
ODE   2%|█                                              |  ETA: 0:05:33
ODE   4%|██                                             |  ETA: 0:05:08
ODE   6%|███                                            |  ETA: 0:04:50
ODE   8%|███▉                                           |  ETA: 0:04:36
ODE  10%|████▉                                          |  ETA: 0:04:23
ODE  12%|█████▉                                         |  ETA: 0:04:11
ODE  15%|██████▉                                        |  ETA: 0:03:59
ODE  17%|███████▉                                       |  ETA: 0:03:49
ODE  19%|████████▊                                      |  ETA: 0:03:39
ODE  21%|█████████▊                                     |  ETA: 0:03:30
ODE  23%|██████████▊                                    |  ETA: 0:03:21
ODE  25%|███████████▊                                   |  ETA: 0:03:12
ODE  27%|████████████▊                                  |  ETA: 0:03

 85.975406 seconds (474.16 M allocations: 15.690 GiB, 1.98% gc time, 9.06% compilation time)
[308.81739933127403, 687107.106657062, 0.130694473172715, 332.7862285876953]


┌ Warning: Skipped marker arg o-.
└ @ Plots /Users/claresinger/.julia/packages/Plots/HcxwM/src/args.jl:1178
